In [125]:
import cv2
import numpy as np
import os

lp_images_path = 'selection/'
lp_images = os.listdir(lp_images_path)

In [126]:
def filtered_lines(lines, filter_vertical):
    filtered = []

    if(lines is None):
        return filtered

    for line in lines:
        x1, y1, x2, y2 = line[0]
        angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        angle_threshold = 5

        if filter_vertical:
            is_vertical = abs(angle) > 90 - angle_threshold or abs(angle) > 90 + angle_threshold
            if is_vertical:
                filtered.append(line)
        else:
            is_horizontal = abs(angle) < angle_threshold or abs(angle - 180) < angle_threshold
            if is_horizontal:
                filtered.append(line)

    return filtered

In [127]:
def plot_edge_detection(threshold_1, threshold_2, hough_threshold, img):
    img_height, img_width, _ = img.shape
    window_title = 'Canny Edge Detection'
    cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_title, 400, 800)

    prep_1 = img.copy()
    prep_1 = cv2.cvtColor(prep_1, cv2.COLOR_BGR2GRAY)
    # rectKern = cv2.getStructuringElement(cv2.MORPH_RECT, (200, 100))
    # prep_1 = cv2.morphologyEx(prep_1, cv2.MORPH_BLACKHAT, rectKern)
    # prep_1 = cv2.equalizeHist(prep_1)

    # canny edge detection
    canny = cv2.Canny(prep_1, threshold_1, threshold_2, apertureSize=3)

    # dilate and erode to close gaps
    prep_2 = canny.copy()
    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    # prep_2 = cv2.morphologyEx(prep_2, cv2.MORPH_CLOSE, kernel)
    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 5))
    # prep_2 = cv2.morphologyEx(prep_2, cv2.MORPH_CLOSE, kernel)

    images = [
        img,
        cv2.cvtColor(prep_1, cv2.COLOR_GRAY2RGB),
        cv2.cvtColor(canny, cv2.COLOR_GRAY2RGB),
        cv2.cvtColor(prep_2, cv2.COLOR_GRAY2RGB),
    ]

    min_lenght_v = int(img_height * 0.2)
    max_gap_v = int(img_height * 0.5)
    min_lenght_h = int(img_width * 0.5)
    max_gap_h = int(img_width * 0.5)

    # hough lines
    hough = img.copy()

    # TODO: define regtion of interest?!


    lines = None
    vertical_lines = cv2.HoughLinesP(prep_2, 1, np.pi/180, hough_threshold, minLineLength=20, maxLineGap=5)
    # vertical_lines = filtered_lines(vertical_lines, True)
    if vertical_lines is not None:
        lines = vertical_lines

    horizontal_lines = cv2.HoughLinesP(prep_2, 1, np.pi/180, hough_threshold, minLineLength=min_lenght_h, maxLineGap=max_gap_h)
    # horizontal_lines = filtered_lines(horizontal_lines, False)
    if horizontal_lines is not None:
        if lines is not None:
            lines = np.concatenate((lines, horizontal_lines), axis=0)
        else:
            lines = horizontal_lines
    

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]

            x_threshold = 50
            y_threshold = 100

            if x1 > x_threshold and x2 < img_width - x_threshold:
                continue

            if y1 > y_threshold and y2 < img_height - y_threshold:
                continue

            # Draw the line on the image
            cv2.line(hough, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
        images.append(hough)
    
    output_image = np.concatenate(images, axis=0)
    cv2.imshow(window_title, output_image)

In [128]:
threshold_window = 'Thresholds'
cv2.namedWindow(threshold_window, cv2.WINDOW_NORMAL)
cv2.resizeWindow(threshold_window, 400, 200)

img = cv2.imread('cropped_more_padding/14790_0_0_11_5_33_25_24.jpg')

def show_detection(val):
    threshold1 = cv2.getTrackbarPos('Threshold 1', threshold_window)
    threshold2 = cv2.getTrackbarPos('Threshold 2', threshold_window)
    hough_threshold = cv2.getTrackbarPos('Hough Threshold', threshold_window)
    plot_edge_detection(threshold1, threshold2, hough_threshold, img)

cv2.createTrackbar('Threshold 1', threshold_window, 250, 500, show_detection)
cv2.createTrackbar('Threshold 2', threshold_window, 250, 500, show_detection)
cv2.createTrackbar('Hough Threshold', threshold_window, 100, 500, show_detection)

show_detection(0)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:866: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:866: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'
